# Byte-pair enconding

In [ ]:
# initialized vocabulary will all unique words in the corpus and their counts. Calculate also pair counts, and update vocabulary iterativelly

def initialize_vocabulary(corpus):
    vocabulary=defaultdict(int)
    charset = set()
    for word in corpus:
        word_with_marker = "_" + word
        characters : list(word_with_marker)
        charset.update(characters)
        tokenized_word = ' '.join(characters)
        vocabulary[tokenized_word] += 1
    return vocabulary, charset